# PyRAT subject ingestion

## Setup

Using local config file (see [01_pipeline](./01_pipeline.ipynb))

In [ ]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session
import datajoint as dj

In [ ]:
db_prefix = dj.config['custom'].get('database.prefix')
print('database Prefix:', db_prefix)

Manual entry

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'tobiasr'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass(prompt='Database password:')
dj.config['custom']['pyrat_client_token'] = getpass.getpass(prompt="Pyrat client token:")
dj.config['custom']['pyrat_user_token'] = getpass.getpass(prompt="Pyrat user token:")
dj.conn()

## Initial check of tables

In [ ]:
# subject.User.delete(); 
# subject.Protocol.delete()
# subject.Line.delete(); 
# subject.Subject.delete()
print('User', len(subject.User()))
print('Protocol', len(subject.Protocol()))
print('Line', len(subject.Line()))
print('Mutation', len(subject.Mutation()))
print('Subject', len(subject.Subject()))
print('SubjectGenotype', len(subject.SubjectGenotype()))

In [ ]:
subject.User()


## Automated ingestion

The function is designed to list all proposed insertions and ask for a confirmation before entered into the schema.

In [ ]:
from adamacs.schemas import subject
from adamacs.ingest.pyrat import PyratIngestion


In [ ]:
# PyratIngestion().ingest_ani
# mal("ROS-1483")
PyratIngestion().ingest_animal("WEZ*")

This function also permits wildcards when querying [the API](https://pyrat.uniklinik-bonn.de/pyrat-test/api/v2/specification/ui#/Listing/get_animals).

In [ ]:
PyratIngestion().ingest_animal("WEZ-8701")

In [ ]:
PyratIngestion().ingest_animal("ROS-1438")

## Confirm entry

In [ ]:
print('User', len(subject.User()))
print('Protocol', len(subject.Protocol()))
print('Line', len(subject.Line()))
print('Mutation', len(subject.Mutation()))
print('Subject', len(subject.Subject()))
print('SubjectGenotype', len(subject.SubjectGenotype()))

In [ ]:
subject.User()

In [ ]:
p = subject.Subject()


In [ ]:
print(p)

In [ ]:
query = subject.Subject() & 'subject = "WEZ-8701"'
l = subject.Subject.fetch('death_date').tolist

In [ ]:
print(l())

In [ ]:
l

In [ ]:
l['subject']

In [ ]:
l()[0]
